In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [2]:
# Load the dataset from 'merged.csv'
data = pd.read_csv('/Users/PRADEEP SHETTY/Desktop/IEEE internship/merged_final.csv')

In [3]:
# Preprocess the data and extract features and labels
X = data[['Avg_activity']].values  # Use 'Avg_activity' column as the feature
y = data['status'].values

In [4]:
# Encode the labels (0 for control and 1 for condition)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [5]:
# One-hot encode the integer labels
y_one_hot = to_categorical(y_encoded)

In [6]:
# Handling timestamps and sorting data
data['timestamp'] = pd.to_datetime(data['date'] + ' ' + data['hour'].apply(lambda x: '{:02d}'.format(x)), format='%Y-%m-%d %H')
data = data.sort_values(by='timestamp')

# Extract timestamps as a numpy array
timestamps = data['timestamp'].values

In [7]:
# Define the sliding window size
window_size = 10  # Choose an appropriate window size based on your data

In [8]:
# Create sequences and labels using the sliding window approach
sequences = []
labels = []

for i in range(len(X) - window_size + 1):
    window_data = X[i : i + window_size]
    sequences.append(window_data)
    labels.append(y_one_hot[i + window_size - 1])

In [9]:
# Convert sequences and labels to numpy arrays
X = np.array(sequences)
y = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Set parameters for the model
sequence_length = window_size  # Define the number of time steps to consider as input
num_features = 1               # Only one feature: 'Avg_activity'
num_classes = 2                # Two classes: condition and control

In [11]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(sequence_length, num_features)))
model.add(Dense(num_classes, activation='softmax'))

In [12]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train the model
num_epochs = 60    # Define the number of epochs for training
batch_size = 32      # Define the batch size
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/60
656/656 [==============================] - 5s 8ms/step - loss: 0.3876 - accuracy: 0.8244 - val_loss: 0.4479 - val_accuracy: 0.7916
Epoch 2/60
656/656 [==============================] - 6s 9ms/step - loss: 0.3861 - accuracy: 0.8237 - val_loss: 0.4539 - val_accuracy: 0.7908
Epoch 3/60
656/656 [==============================] - 6s 9ms/step - loss: 0.3837 - accuracy: 0.8233 - val_loss: 0.4528 - val_accuracy: 0.7954
Epoch 4/60
656/656 [==============================] - 5s 8ms/step - loss: 0.3838 - accuracy: 0.8253 - val_loss: 0.4421 - val_accuracy: 0.7958
Epoch 5/60
656/656 [==============================] - 4s 6ms/step - loss: 0.3796 - accuracy: 0.8294 - val_loss: 0.4423 - val_accuracy: 0.7962
Epoch 6/60
656/656 [==============================] - 5s 8ms/step - loss: 0.3782 - accuracy: 0.8297 - val_loss: 0.4444 - val_accuracy: 0.7928
Epoch 7/60
656/656 [==============================] - 5s 7ms/step - loss: 0.3769 - accuracy: 0.8284 - val_loss: 0.4563 - val_accuracy: 0.7920
Epoch 

656/656 [==============================] - 6s 9ms/step - loss: 0.3064 - accuracy: 0.8637 - val_loss: 0.4548 - val_accuracy: 0.8088
Epoch 59/60
656/656 [==============================] - 6s 9ms/step - loss: 0.3070 - accuracy: 0.8643 - val_loss: 0.4431 - val_accuracy: 0.8112
Epoch 60/60
656/656 [==============================] - 4s 6ms/step - loss: 0.3034 - accuracy: 0.8656 - val_loss: 0.4465 - val_accuracy: 0.8132


In [17]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

164/164 [==============================] - 0s 3ms/step - loss: 0.4465 - accuracy: 0.8132
Test Loss: 0.44645029306411743, Test Accuracy: 0.8131554126739502


In [15]:
# Making Predictions and Printing Actual and Predicted Values
y_pred_one_hot = model.predict(X_test)
y_pred = np.argmax(y_pred_one_hot, axis=1)
predicted_labels = label_encoder.inverse_transform(y_pred)
actual_labels = label_encoder.inverse_transform(np.argmax(y_test, axis=1))

print("Timestamp\t\t\tActual Status\t\tPredicted Status")
for i, timestamp in enumerate(timestamps[-len(X_test):]):
    print(f"{timestamp}\t{actual_labels[i]}\t\t\t{predicted_labels[i]}")

164/164 [==============================] - 1s 3ms/step
Timestamp			Actual Status		Predicted Status
2004-06-09T03:00:00.000000000	1			0
2004-06-09T04:00:00.000000000	0			0
2004-06-09T05:00:00.000000000	0			0
2004-06-09T06:00:00.000000000	1			1
2004-06-09T07:00:00.000000000	1			1
2004-06-09T08:00:00.000000000	0			0
2004-06-09T09:00:00.000000000	0			0
2004-06-09T10:00:00.000000000	0			1
2004-06-09T11:00:00.000000000	0			0
2004-06-09T12:00:00.000000000	0			0
2004-06-09T13:00:00.000000000	0			0
2004-06-09T14:00:00.000000000	0			0
2004-06-09T15:00:00.000000000	0			0
2004-06-09T16:00:00.000000000	0			0
2004-06-09T17:00:00.000000000	1			0
2004-06-09T18:00:00.000000000	0			0
2004-06-09T19:00:00.000000000	0			0
2004-06-09T20:00:00.000000000	0			0
2004-06-09T21:00:00.000000000	0			0
2004-06-09T22:00:00.000000000	1			1
2004-06-09T23:00:00.000000000	0			1
2004-06-10T00:00:00.000000000	1			0
2004-06-10T01:00:00.000000000	0			0
2004-06-10T02:00:00.000000000	0			0
2004-06-10T03:00:00.000000000	0			0
2

2005-09-26T11:00:00.000000000	0			1
2005-09-26T12:00:00.000000000	1			0
2005-09-26T13:00:00.000000000	0			0
2005-09-26T14:00:00.000000000	0			0
2005-09-26T15:00:00.000000000	0			0
2005-09-26T16:00:00.000000000	0			0
2005-09-26T17:00:00.000000000	1			1
2005-09-26T18:00:00.000000000	1			0
2005-09-26T19:00:00.000000000	0			0
2005-09-26T20:00:00.000000000	1			0
2005-09-26T21:00:00.000000000	1			1
2005-09-26T22:00:00.000000000	0			0
2005-09-26T23:00:00.000000000	0			0
2005-09-27T00:00:00.000000000	0			0
2005-09-27T01:00:00.000000000	0			0
2005-09-27T02:00:00.000000000	0			0
2005-09-27T03:00:00.000000000	1			1
2005-09-27T04:00:00.000000000	1			0
2005-09-27T05:00:00.000000000	0			0
2005-09-27T06:00:00.000000000	1			0
2005-09-27T07:00:00.000000000	0			0
2005-09-27T08:00:00.000000000	1			1
2005-09-27T09:00:00.000000000	1			1
2005-09-27T10:00:00.000000000	0			0
2005-09-27T11:00:00.000000000	1			1
2005-09-27T12:00:00.000000000	1			0
2005-09-27T13:00:00.000000000	1			0
2005-09-27T14:00:00.00000000